In [1]:
# pip install dotenv
# %pip install pyTelegramBotAPI
# %pip install git+https://github.com/huggingface/transformers.git
# %pip install accelerate


In [2]:
import os
from dotenv import load_dotenv
import telebot
import torch
from transformers import pipeline

In [3]:
# Load the .env file
load_dotenv("environment.env")

# Access the BOT_TOKEN variable
BOT_TOKEN = os.getenv("BOT_TOKEN")
bot = telebot.TeleBot(BOT_TOKEN)

In [4]:
@bot.message_handler(commands=['start', 'hello', 'hi'])
def send_welcome(message):
    bot.reply_to(message, "hi, how can i help?")


In [ ]:
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16,
    device=0 if torch.cuda.is_available() else -1 
)

@bot.message_handler(func=lambda msg: True)
def echo_all(message):
    prompt = pipe.tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "You are a friendly chatbot who always responds in the style of an expert"},
            {"role": "user", "content": message.text},
        ],
        tokenize=False,
        add_generation_prompt=True
    )
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

    generated_text = outputs[0]["generated_text"]
    assistant_response = generated_text.split("<|assistant|>")[-1].strip()

    bot.reply_to(message, assistant_response)

# Run the bot
bot.infinity_polling()

Device set to use cuda:0
